# 演習課題6回目

12月1日(金)

csvファイル(data_cv.csv)のデータを9次多項式でRidge回帰することを考える．
$\alpha$の値を$10^{-2}, 10^{-1}, 10^{0}, 10^{1}, 10^{2}$と変えたときのk-分割交差検証における訓練データとテストデータの評価の推移を横軸を$\alpha$の対数スケールとして示せ．

## セットアップ

In [1]:
# Pythonが実行時のバージョンを満たしているか確認する
import sys
assert sys.version_info >= (3, 9, 7)
# 必要なモジュールのインポート
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
# サードパーティ製ライブラリが実行時のバージョンを満たしているか確認する
assert np.__version__ >= '1.20.3'
assert pd.__version__ >= '1.3'
assert sklearn.__version__ >= '0.24'
# データの保存場所の定義
DATA_ROOT_PATH = Path('data', '8')
# matplotlib の日本語対応
# plt.rcParams['font.family'] = "MS Gothic"


## データの取得

In [2]:
data = pd.read_csv(DATA_ROOT_PATH / 'data_cv.csv').to_numpy()
data[:5]


array([[  1.71236474,  19.0100227 ],
       [ -1.9797037 , -59.42523313],
       [ -0.28481095,   2.62715867],
       [ -1.90773524, -51.7854505 ],
       [ -1.82626875, -40.10011762]])

In [3]:
X, y = data[:, 0].reshape(-1, 1), data[:, 1]
X[:5]


array([[ 1.71236474],
       [-1.9797037 ],
       [-0.28481095],
       [-1.90773524],
       [-1.82626875]])

## データの加工

9次多項式で回帰するためにデータを加工する

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=9, include_bias=False)),
])

X_poly = pipeline.fit_transform(X)
X_poly[:5]


array([[ 1.71236474e+00,  2.93219300e+00,  5.02098390e+00,
         8.59775578e+00,  1.47224938e+01,  2.52102793e+01,
         4.31691933e+01,  7.39214045e+01,  1.26580406e+02],
       [-1.97970370e+00,  3.91922672e+00, -7.75890762e+00,
         1.53603381e+01, -3.04089181e+01,  6.02006475e+01,
        -1.19179444e+02,  2.35939986e+02, -4.67091263e+02],
       [-2.84810947e-01,  8.11172758e-02, -2.31030882e-02,
         6.58001243e-03, -1.87405957e-03,  5.33752682e-04,
        -1.52018607e-04,  4.32965635e-05, -1.23313353e-05],
       [-1.90773524e+00,  3.63945374e+00, -6.94311415e+00,
         1.32456235e+01, -2.52691428e+01,  4.82068341e+01,
        -9.19658762e+01,  1.75446543e+02, -3.34705552e+02],
       [-1.82626875e+00,  3.33525756e+00, -6.09107665e+00,
         1.11239430e+01, -2.03153094e+01,  3.71012148e+01,
        -6.77567893e+01,  1.23742107e+02, -2.25986343e+02]])

## Ridge モデルの訓練とk-分割交差検証

In [5]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_validate

def ridge_reg_cross_val(X, y, alpha_values):
    """Ridgeモデルの訓練とk-分割交差検証

    Args:
        X: データ
        y: ラベル
        alpha_values: 検証するα値の配列

    Returns:
        tuple: 各α値に対する訓練スコアとテストスコアが組になったタプル
    """
    train_scores = np.zeros(len(alpha_values))
    test_scores = np.zeros(len(alpha_values))

    for i, alpha in enumerate(alpha_values):
        ridge = Ridge(alpha=alpha)
        cv_results = cross_validate(ridge, X, y, cv=5, return_train_score=True)
        train_scores[i] = np.mean(cv_results['train_score'])
        test_scores[i] = np.mean(cv_results['test_score'])

    return train_scores, test_scores


In [6]:
alpha_values = [10 ** a for a in range(-2, 3)]
train_scores, test_scores = ridge_reg_cross_val(X_poly, y, alpha_values)
train_scores


array([0.78431854, 0.78400582, 0.78254877, 0.78046121, 0.76664655])

In [7]:
def plot_cross_val(ax, train_scores, test_scores, alpha_values):
    ax.plot(alpha_values, train_scores, label='Train')
    ax.plot(alpha_values, test_scores, label='Test')
    ax.set_xlabel(r"$\alpha$")
    ax.set_xscale('log')
    ax.set_ylabel('Score')
    ax.grid()
    ax.legend()

    return ax


## 評価の推移の描画

In [8]:
max_test_score = test_scores.max()

fig, ax = plt.subplots()
ax = plot_cross_val(ax, train_scores, test_scores, alpha_values)
ax.axhline(max_test_score, c='k', linestyle=':')
ax.text(10, max_test_score, str(round(max_test_score, 3)))
plt.show()
